In [54]:
#setup
%load_ext autoreload
%autoreload 2


import os
import numpy as np
import pandas as pd

os.chdir('C:/repos/E-OBS-SWB2')

from Python.RechargeCalc import RechargeCalc

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
startyear = 2014
endyear = 2018
cell_area = 100*100 #m2
#Path to the SWB2 output
swb2path = "./Data/dati_articolo_modelmi_25/VersioneFINALE_net_infiltration.nc"
#Path to the input .csv files folder
inputpath = "./Data/dati_articolo_modelmi_25/160SP"

r = RechargeCalc(startyear, endyear, cell_area, uniqueid = 'indicatore', nSP = 148, urb = False)
r.load_inputfiles(swb2path, inputpath)

SP1 = 90   #days, 01/01 - 30/03
SP2 = 76   #days, 01/04 - 12/06
SP3 = 92   #days, 13/06 - 15/09
SP4 = 107  #days, 16/09 - 31/12
SPs = [SP1, SP2, SP3, SP4]

r.meteoricR(SPs)

coeffs = {
    'E': 0.3,  #Irrigation technique efficiency
    'R': 0.05, #Residual runoff
    'RISP': 1, #1 - fraction of water saved by a change of irrigation technique
    'P': 1     #Percentage of the cell covered by the irrigation
    }

# sc = [0, 0.15, 0.30, 0.5]
sc = [0.3]
for coeff in sc:
    r.irrigationR(coeffs)
    r.modify_recharge('recharge','rirr',coeff=(1-coeff),cond='null')
    r.totalR()
    r.export('recharge','rtot', outpath = os.path.join(os.getcwd(), 'Data','dati_articolo_modelmi_25'), outname=f'rtot_IRR{int(coeff*100)}_2050',
             withcoord=True, coordpath = os.path.join(os.getcwd(), 'Data','dati_articolo_modelmi_25', '160SP', 'coord.csv'))


Loading the input files
-----------------------
indicatori file found
ricarica_irrigua file found
Meteoric recharge dataframe creation
------------------------------------
Performing the sum of net_infiltration over the stress periods provided
Output unit measure: ms
End of the procedure
Replicating columns to create a dataframe of 148 SPs
Elapsed time: 53.06 s
Irrigation recharge dataframe creation
--------------------------------------
Elapsed time: 7.05 s
Total recharge dataframe creation
---------------------------------
Elapsed time: 3.14 s
67.31634283065796 s


In [ ]:
# obtain shapefiles from the saved csv


import geopandas as gpd
from shapely.geometry import Point

test = pd.read_csv(os.path.join(os.getcwd(), 'Data','dati_articolo_modelmi_25','rtot_IRR0_2050.csv'))
points = [Point(x,y) for x,y in zip(test.X,test.Y)]
geodf = gpd.GeoDataFrame(test, geometry=points)
geodf.to_file('rtot_IRR0_2050.shp', driver='ESRI Shapefile', engine = 'fiona')

# it takes 11 minutes each